### Step1 - 导入相关包 & 初始化设置

In [ ]:
import os, sys

import bs4
import numpy as np

from langchain_community.document_loaders import WebBaseLoader
# https://reference.langchain.com/python/langchain_text_splitters/
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 默读取当前目录下的 .env 文件, 可以通过 dotenv_path 来修改
from dotenv import load_dotenv
root_dir = os.path.dirname(os.getcwd())
load_dotenv(dotenv_path=os.path.join(root_dir, ".env"))

# 记录日志
from loguru import logger
logger.remove()
logger.add(sys.stderr, level=os.getenv("LOG_LEVEL"))

# 设定 USER_AGENT 环境变量, 否则会提示未设置
os.environ["USER_AGENT"] = "Mozilla/5.0 (Linux; Android 16; Pixel 9) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.12.45 Mobile Safari/537.36"

### Step2 - 抓取 Web 网站

In [4]:
# 利用 bs4 抓取 web
# 利用 WebBaseLoader 来读取 bs 对象
page_url = "https://docs.langchain.com/oss/python/langchain/overview"
# 利用 SoupStrainer 来进行选择性解析网页
bs4_strainer = bs4.SoupStrainer()

loader = WebBaseLoader(
    web_paths=(page_url,),
    bs_kwargs={"parse_only": bs4_strainer},
)

docs = loader.load()

logger.debug(f"整个文档对象的类型: {type(docs)}")
logger.debug(f"每个文档对象的类型: {type(docs[0])}")
# 文档内容为 docs[0].page_content
logger.debug(f"网页页数: {len(docs)}")
logger.debug(f"网页内容: {docs[0].page_content}")
logger.debug(f"网页元数据: {docs[0].metadata}")

2025-12-07 14:25:21.663 | DEBUG    | __main__:<module>:14 - 整个文档对象的类型: <class 'list'>
2025-12-07 14:25:21.664 | DEBUG    | __main__:<module>:15 - 每个文档对象的类型: <class 'langchain_core.documents.base.Document'>
2025-12-07 14:25:21.665 | DEBUG    | __main__:<module>:17 - 网页页数: 1
2025-12-07 14:25:21.665 | DEBUG    | __main__:<module>:18 - 网页内容: LangChain overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangChain + LangGraphSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain overviewLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonOverviewGet startedInstallQuickstartPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingStructured outputMiddlewareOverviewBuilt-in middlewareCustom middlewareAdvanced usageGuardrailsRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryAgent develop

### Step3 - 切分文档

In [5]:
# 创建一个文本切分器, 将文档切分为块(chunk)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
# 切分指定文档
# all_chunks 是一个列表, 每个元素是一个 Document 对象
all_chunks = text_splitter.split_documents(docs)
# Document 内容为 all_chunks[0].page_content
logger.debug(f"切分后每个块元数据: {all_chunks[0].metadata}")

2025-12-07 14:25:43.522 | DEBUG    | __main__:<module>:11 - 切分后每个块元数据: {'source': 'https://docs.langchain.com/oss/python/langchain/overview', 'title': 'LangChain overview - Docs by LangChain', 'language': 'en', 'start_index': 0}


### Step4 - 将切分后的文档块向量化

In [6]:
# 创建一个嵌入式模型, 这里使用 Ollama 嵌入模型来进行文本向量化
embed_model = OllamaEmbeddings(
    model=os.getenv("OLLAMA_EMB_MODEL"),
    base_url=os.getenv("OLLAMA_BASE_URL"),
)
# 可以通过 embed_query() 方法将单个文本向量化
# 可以通过 embed_documents() 方法将多个文本向量化
logger.debug(f"单个文本经过向量化后的维度: {len(embed_model.embed_query(all_chunks[0].page_content))}")
logger.debug(f"多个文本经过向量化后的形状: {np.array(embed_model.embed_documents(["doc_test1", "doc_test2"])).shape}")

2025-12-07 14:25:56.798 | DEBUG    | __main__:<module>:8 - 单个文本经过向量化后的维度: 2560
2025-12-07 14:25:57.007 | DEBUG    | __main__:<module>:9 - 多个文本经过向量化后的形状: (2, 2560)


### Step5 - 保存向量化后的数据

In [7]:
# 需要创建向量数据库, 这里使用 Chroma 来进行存储
vector_db = Chroma(
    # 集合名字
    # 可以通过创建一个 chromadb.PersistentClient(Path) 对象, 并通过 list_collections() 方法来获得所有集合的名字
    collection_name="example_collection",
    embedding_function=embed_model,
    persist_directory="./chroma_db",
    # 可选配置, 涉及如何计算相似度, Chroma 默认使用的是 L2 距离
    # https://docs.trychroma.com/docs/collections/configure#hnsw-index-configuration
    # https://reference.langchain.com/python/integrations/langchain_chroma/
    # https://reference.langchain.com/python/integrations/langchain_chroma/#langchain_chroma.Chroma.as_retriever
    collection_metadata={"hnsw:space": "l2"}
)
# 将切分好的文档添加到向量数据库中
ids = vector_db.add_documents(all_chunks)
logger.info(f"已成功添加 {len(ids)} 个文档分块的记录")

2025-12-07 14:25:59.992 | INFO     | __main__:<module>:16 - 已成功添加 7 个文档分块的记录
